In [ ]:
                                                       
library(tidyverse) # metapackage of all tidyverse packages

# Import necessary R packages Singular Spectrum Analysis and prediction

library(Rssa)
library(readxl)
library(stringr)



In [ ]:
# This function load the data

importData <- function(path){

  data <- read_csv(path)
          return(data)
    
    }
  





In [ ]:
# This function perform Singular Spectrum Analysis
ssa_ <- function(path, export){
    
    # Import the data
    data_ <- importData(path)
    
    # Decompsition of the data
    decomp<- ssa(data_, L = 12)
    
    # Deccomposed data is a matrix, getting the plot of the Eigenvalues, 
    # Eigenvectors, Pairs-eigenvectors for these matrix 
    # And correlation matrix compare with the origanl data 
    
    Eigenvalues <- plot(decomp) # Eigenvalues
    Eigenvectors <- plot(decomp, type = "vectors") # Eigenvectors
    Pairs_Eigenvectors <- plot(decomp, type = "paired") # Pairs of eigenvectors
    w_correlation <- plot(wcor(decomp)) # w-correlation matrix plot
    # w-correlation matrix for reconstruction
    w_correlation_ <- plot(wcor(decomp, groups = list(c(1,4), c(2,3), c(5,6))))
    
    
    
    # Reconstruction of the decomsed data
    recon <- reconstruct(decomp, groups = list(c(1,4), c(2,3), c(5,6)))
    
    
    # Residual
    residual <- residuals(recon)
    
    
    # Exporting the reconstruction and residual matrix
    recon_export <- cbind("reconstructed",export,".csv")
    
    write.csv(recon$F1, str_c(recon_export, collapse = "_"))
    
    residual_export <- cbind("residual", export, ".csv")
    
    write.csv(residual, str_c(residual_export, collapse = "_"))
    
    result <- list(recon$F1,recon, residual,Eigenvalues,Eigenvectors,
                   Pairs_Eigenvectors, w_correlation, w_correlation_)
    
    return(result)
    
    }




In [ ]:
# This function is to plot the output found from SSA analysis
plot_ssa <- function(path, export){
    
    # Import data and necessary variables
    data <- ssa_(path,export)
    
    original <- importData(path)
    
    re_tem<- cbind("reconstructed",export,".csv")
    
    reconstruction <- importData(str_c(re_tem, collapse = "_"))
    
    resd <- cbind("residual", export, ".csv")
    
    residual <- importData(str_c(resd, collapse = "_"))
    
    Eigenvalues <- data[4]
    
    Eigenvectors <- data[5]
    
    Pairs_Eigenvectors <- data[6]
    
    w_correlation <- data[7]
    
    w_correlation_ <- data[8]
    
    
    
    
    # Decomposition into trend + seasonality and noise
    reconstruction_plot <- plot(reconstruction, main = "Reconstructed", col = 'yellow')
    
    # Plotting original data
    original_plot <- plot(original, main = 'original', col = 'green')
    
    # Plotting the residual data
    residual_plot <- plot(residual, main = 'residual', col = 'blue')
    
    
    result <- list(Eigenvalues, Eigenvectors,Pairs_Eigenvectors, w_correlation, 
                   w_correlation_, original_plot, reconstruction_plot, residual_plot )
    
    return(result)
}




In [ ]:
#Defining the path to load the data

vd <- "../input/distance-vid0/DistanceNmotion_datavideo0.csv"






# Show the first 6 rows of the data
head(importData(vd))

In [ ]:
# Get the output graphs for the specified data
plot_ssa(vd, "Singular_spectrum_analysis_of_distance")

In [ ]:
# The graph reconstructed time series was smooth (noise free) 
# but shaped slightly concave or convex (the respective original plot is greatly concave or convex respectively) 

In [ ]:
# Gap filling SSA analysis
# Gapfilling deals with missing data
gapFilling <- function(path){
    
    data <- importData(path)
    
    F <- importData(path)
    
    # m is the half of the number of columns of the imported data
    m <- nrow(data)/2
    
    # n is equal to the length of the 5% data length
    n <- as.integer(m*2/10/2)

    
    n1 <- m-n
    n2 <- m+n
    
    # 10% data from the middle is setting to the missing data 
    F[n1:n2,] <- NA
    
    # SSA step 1 and 2
    decomposition <- ssa(F, L = 11)
    
    # Applying gap-filling algorithm for alpha = 0
    gapfill0 <- gapfill(decomposition, groups = list(c(1, 4)), method = "sequential",
                        alpha = 0, base = "reconstructed")
    
    
    # Applying gap-filling algorithm for alpha = 1
    gapfill1 <- gapfill(decomposition, groups = list(c(1, 4)), method = "sequential",
                        alpha = 1, base = "reconstructed")
    
    # Applying gap-filling algorithm for defalut alpha
    gapfill_ <- gapfill(decomposition, groups = list(c(1, 4)), method = "sequential",
                        base = "reconstructed")
    
    # Plotting
    data_plot <- plot(F, type = 'l', col = "black")
        
    gapfill0_plot <- lines(gapfill0, col = "blue", lwd = 2)
    gapfill1_plot <- lines(gapfill1, col = "green", lwd = 2)
    gapfill_plot <- lines(gapfill_, col = "red", lwd = 2)
    
    result <- list(gapfill0, gapfill1, gapfill_, gapfill0_plot,gapfill1_plot, gapfill_plot)
    
    return(result)
    }

In [ ]:
# Calling the function gapFilling for the rainfallIndia data
gapFilling(vd)